**Unzip files**

In [1]:
import zipfile
from google.cloud import storage
!gsutil ls gs://$BUCKET

gs://dataproc-staging-us-central1-778328429038-q1hvilaw/
gs://dataproc-temp-us-central1-778328429038-h8hshfg7/
gs://enem2020-bucket/


**Installing Spark using Colab**

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
from pyspark.sql.functions import max
from pyspark.sql.functions import desc
spark = SparkSession.builder.appName("Dados Enem2020").getOrCreate()
#master("local[16]").

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/20 01:41:21 INFO SparkEnv: Registering MapOutputTracker
23/08/20 01:41:21 INFO SparkEnv: Registering BlockManagerMaster
23/08/20 01:41:22 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
23/08/20 01:41:22 INFO SparkEnv: Registering OutputCommitCoordinator


In [3]:
df = spark.read.csv('gs://enem2020-bucket/microdados_enem_2020 (2)/DADOS/MICRODADOS_ENEM_2020.csv',header = True, inferSchema=True, sep = ';')

In [4]:
df.rdd.getNumPartitions()

16

In [5]:
df.write.parquet('gs://enem2020-bucket/microdados_enem_2020 (2)/parket/MICRODADOS_ENEM_2020.parket')

23/08/20 01:42:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
pardf=spark.read.parquet("gs://enem2020-bucket/microdados_enem_2020 (2)/parket/MICRODADOS_ENEM_2020.parket")

In [7]:
pardf.show(10)

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|T

Quantos alunos não quiseram declarar a cor/raça em 2020 (Entenda a opção “não declarado” nessa pergunta)?

In [9]:
from pyspark.sql.functions import col
pardf.filter(col("TP_COR_RACA") == 0).count()

116883

Qual é o número de alunos do Sexo Feminino que estudaram em escola no estado de São Paulo?

In [10]:
pardf.createGlobalTempView("pardfview")

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used


In [11]:
spark.sql("SELECT * FROM global_temp.pardfview WHERE TP_SEXO == 'F' AND SG_UF_ESC == 'SP' ").count()

91480

Quantos alunos do sexo feminino que estudaram em escola no estado do Rio Grande do Sul possuem EXATAMENTE uma geladeira em casa?

In [12]:
spark.sql("SELECT * FROM global_temp.pardfview WHERE TP_SEXO == 'F' AND SG_UF_ESC == 'RS' AND Q012 == 'B' ").count()

18680

Qual é o segundo estado brasileiro (considere a coluna SG_UF_ESC) em que estudaram mais alunos no ENEM 2020?

In [13]:
spark.sql("SELECT SG_UF_ESC, count(TP_SEXO) c FROM global_temp.pardfview GROUP BY SG_UF_ESC ORDER BY c DESC ").show(28)

+---------+-------+
|SG_UF_ESC|      c|
+---------+-------+
|     null|4878540|
|       SP| 167183|
|       CE| 104981|
|       MG|  71614|
|       RJ|  62583|
|       PE|  47583|
|       BA|  47235|
|       PR|  43060|
|       GO|  39769|
|       RS|  36478|
|       PA|  33109|
|       AM|  30120|
|       SC|  27373|
|       MA|  26104|
|       DF|  19840|
|       PB|  19760|
|       ES|  18419|
|       PI|  15738|
|       MT|  14674|
|       RN|  14327|
|       AL|  13976|
|       MS|  11851|
|       SE|  10878|
|       RO|   9927|
|       TO|   8457|
|       AP|   3565|
|       AC|   3512|
|       RR|   2453|
+---------+-------+



Qual é o estado brasileiro (considere a coluna SG_UF_ESC) possui o menor número de alunos cuja mãe possui ensino superior completo?

In [14]:
spark.sql("SELECT SG_UF_ESC, count(NU_INSCRICAO) i FROM global_temp.pardfview WHERE (Q002 == 'F') GROUP BY SG_UF_ESC ORDER BY i DESC ").show(30)

+---------+------+
|SG_UF_ESC|     i|
+---------+------+
|     null|418795|
|       SP| 32542|
|       MG| 10333|
|       RJ|  9229|
|       PR|  6046|
|       RS|  5175|
|       BA|  5040|
|       CE|  4159|
|       GO|  4150|
|       PE|  4089|
|       DF|  3735|
|       SC|  3592|
|       PA|  3552|
|       MA|  2302|
|       AM|  2224|
|       MT|  2206|
|       PB|  2038|
|       ES|  1966|
|       MS|  1823|
|       RN|  1712|
|       PI|  1641|
|       AL|  1197|
|       TO|  1114|
|       RO|  1002|
|       SE|   936|
|       AP|   494|
|       AC|   491|
|       RR|   425|
+---------+------+



Qual é o segundo estado brasileiro (considere a coluna SG_UF_ESC) que possui o maior número de pessoas na faixa “entre 26 e 30 anos”?


In [15]:
spark.sql("SELECT SG_UF_ESC, count(NU_INSCRICAO) i FROM global_temp.pardfview WHERE (TP_FAIXA_ETARIA == 11) GROUP BY SG_UF_ESC ORDER BY i DESC ").show(30)

+---------+------+
|SG_UF_ESC|     i|
+---------+------+
|     null|567071|
|       CE|  1880|
|       AM|   828|
|       SP|   580|
|       GO|   518|
|       PA|   479|
|       PE|   461|
|       RJ|   456|
|       BA|   450|
|       MG|   432|
|       PR|   221|
|       RO|   215|
|       PB|   201|
|       RS|   195|
|       SE|   183|
|       MA|   160|
|       PI|   154|
|       TO|   150|
|       RN|   121|
|       ES|   117|
|       AL|   113|
|       MS|   107|
|       MT|    91|
|       SC|    82|
|       DF|    80|
|       AP|    42|
|       AC|    35|
|       RR|    20|
+---------+------+



Qual é o estado brasileiro (considere a coluna SG_UF_ESC) que possui o TERCEIRO maior número de alunos cuja residência possui PELO MENOS 2 banheiros?

In [16]:
spark.sql("SELECT SG_UF_ESC, count(NU_INSCRICAO) i FROM global_temp.pardfview WHERE (Q008 == 'C' OR Q008 == 'D' OR Q008 == 'E' ) GROUP BY SG_UF_ESC ORDER BY i DESC ").show(30)

+---------+-------+
|SG_UF_ESC|      i|
+---------+-------+
|     null|1151573|
|       SP|  83502|
|       MG|  30912|
|       RJ|  24367|
|       PR|  20938|
|       CE|  15533|
|       GO|  14562|
|       RS|  14470|
|       SC|  13104|
|       BA|  13057|
|       PE|  11518|
|       DF|  10231|
|       PA|   8055|
|       ES|   7185|
|       PB|   5978|
|       AM|   5843|
|       MT|   5826|
|       RN|   5439|
|       MA|   5373|
|       MS|   4995|
|       PI|   3953|
|       AL|   3724|
|       SE|   2966|
|       RO|   2543|
|       TO|   2397|
|       RR|    973|
|       AP|    937|
|       AC|    827|
+---------+-------+



Quantos alunos do sexo feminino se autodeclararam pretos?


In [17]:
spark.sql("SELECT NU_INSCRICAO FROM global_temp.pardfview WHERE (TP_SEXO == 'F' AND TP_COR_RACA == 2) ").count()

444346

Quantos alunos estrangeiros fizeram o ENEM 2020?


In [18]:
spark.sql("SELECT NU_INSCRICAO FROM global_temp.pardfview WHERE TP_NACIONALIDADE == 3 ").count()

8036

Qual é a diferença da nota média em matemática dos alunos que estudaram o ensino médio em escola pública e em escola privada?


In [19]:
publica = spark.sql("SELECT AVG(NU_NOTA_MT) FROM global_temp.pardfview WHERE TP_ESCOLA == 2 ").show()

+------------------+
|   avg(NU_NOTA_MT)|
+------------------+
|499.52498038446714|
+------------------+



In [20]:
privada = spark.sql("SELECT AVG(NU_NOTA_MT) FROM global_temp.pardfview WHERE TP_ESCOLA == 3 ").show()

+-----------------+
|  avg(NU_NOTA_MT)|
+-----------------+
|610.6393569797276|
+-----------------+



In [21]:
difnotasmat = 610.639 - 499.52

Qual é o estado brasileiro (considere a coluna SG_UF_ESC) que possui o maior número de alunos do sexo feminino indígenas?

In [22]:
spark.sql("SELECT SG_UF_ESC, count(NU_INSCRICAO) i FROM global_temp.pardfview WHERE (TP_SEXO == 'F' AND TP_COR_RACA == 5) GROUP BY SG_UF_ESC ORDER BY i DESC ").show(30)

+---------+-----+
|SG_UF_ESC|    i|
+---------+-----+
|     null|18081|
|       AM|  398|
|       CE|  380|
|       SP|  298|
|       PE|  249|
|       BA|  200|
|       PA|  144|
|       MG|  144|
|       MS|  118|
|       RJ|  118|
|       PB|  110|
|       AL|   88|
|       GO|   83|
|       RR|   71|
|       TO|   69|
|       ES|   68|
|       PI|   59|
|       PR|   55|
|       MA|   54|
|       RO|   53|
|       DF|   42|
|       MT|   39|
|       SE|   38|
|       RS|   32|
|       SC|   31|
|       RN|   23|
|       AC|   16|
|       AP|   16|
+---------+-----+



Qual é a diferença entre o número de alunos cujo pai possui pós-graduação completa e o número de alunos cuja mãe possui pós-graduação completa?

In [23]:
DiformacaoPaiMae = spark.sql("SELECT NU_INSCRICAO FROM global_temp.pardfview WHERE (Q001 == 'G') ").count() - spark.sql("SELECT NU_INSCRICAO FROM global_temp.pardfview WHERE (Q002 == 'G') ").count()

In [24]:
print(DiformacaoPaiMae)

-194717


Quantos alunos cuja residência possui ATÉ 2 carros estudaram na região NORDESTE do Brasil?

In [25]:
spark.sql("SELECT * FROM global_temp.pardfview WHERE ( ( Q010 == 'B' OR Q010 == 'C') AND ( SG_UF_ESC == 'MA' OR SG_UF_ESC == 'PI' OR SG_UF_ESC == 'CE' OR SG_UF_ESC == 'RN' OR SG_UF_ESC == 'PB' OR SG_UF_ESC == 'PE' OR SG_UF_ESC == 'AL' OR SG_UF_ESC == 'SE' OR SG_UF_ESC == 'BA') )  ").count()

74533

Quantos alunos que estudaram em escolas em zona rural possuem internet em casa?

In [26]:
spark.sql("SELECT NU_INSCRICAO FROM global_temp.pardfview WHERE (TP_LOCALIZACAO_ESC == 2 AND Q025 == 'B') ").count()

18979

Qual é o nome do município que contém a SEGUNDA maior quantidade total de inscritos no ENEM 2020?

In [27]:
spark.sql("SELECT NO_MUNICIPIO_PROVA, count(NU_INSCRICAO) i FROM global_temp.pardfview GROUP BY NO_MUNICIPIO_PROVA ORDER BY i DESC ").show(30)

+------------------+------+
|NO_MUNICIPIO_PROVA|     i|
+------------------+------+
|         S�o Paulo|250475|
|    Rio de Janeiro|156727|
|          Bras�lia|116932|
|            Manaus|114037|
|         Fortaleza|108508|
|          Salvador|100075|
|    Belo Horizonte| 98720|
|             Bel�m| 94499|
|          S�o Lu�s| 82212|
|            Recife| 72652|
|           Goi�nia| 61118|
|          Curitiba| 54272|
|          Teresina| 54021|
|             Natal| 44580|
|       Jo�o Pessoa| 44536|
|            Macei�| 42008|
|      Porto Alegre| 37934|
|      Campo Grande| 36186|
|           Aracaju| 34557|
|            Macap�| 31854|
|       Porto Velho| 29980|
|         Guarulhos| 29092|
|            Cuiab�| 29007|
|        Ananindeua| 28804|
|  Feira de Santana| 28561|
|        Rio Branco| 27481|
|    Campina Grande| 26825|
|        Uberl�ndia| 25306|
|          Campinas| 23700|
|     Montes Claros| 21445|
+------------------+------+
only showing top 30 rows

